In [ ]:
import os
os.chdir("../")

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt
from aeon.distances import pairwise_distance
from IPython.display import display, clear_output
from time import time

In [ ]:
from storage.har_datasets import HARTHDataset, UCI_HARDataset
from s3ts.api.ts2sts import compute_medoids
from s3ts.api.encodings import compute_DM
from storage.label_mappings import *

In [ ]:
ds = UCI_HARDataset("./datasets/UCI-HAR/", split="train", wsize=64, normalize=True)

In [ ]:
def obtain_barycenter(dataset, n_random=100, normalize=False):
    window_id, window_lb = dataset.getSameClassWindowIndex()

    np.random.seed(0)

    selected = np.empty((np.unique(window_lb).shape[0], dataset.wsize, dataset.STS.shape[-1]))

    for i, c in enumerate(np.unique(window_lb)):
        # get the random windows for the class c

        rw = np.random.choice(window_id[window_lb == c].reshape(-1), n_random)

        ts, cs = dataset.sliceFromArrayOfIndices(rw)

        km = TimeSeriesKMeans(n_clusters=1, verbose=True, random_state=1, metric="dtw", n_jobs=-1)
        km.fit(ts)

        selected[i] = km.cluster_centers_[0]
    
    return selected

In [ ]:
# barycenters = obtain_barycenter(ds, 500)

In [ ]:
def obtain_medoids(dataset, n_random=100, normalize=False):
    window_id, window_lb = dataset.getSameClassWindowIndex()

    np.random.seed(0)

    selected_w = []
    selected_c = []

    for i, c in enumerate(np.unique(window_lb)):
        # get the random windows for the class c

        rw = np.random.choice(window_id[window_lb == c].reshape(-1), n_random)

        ts, cs = dataset.sliceFromArrayOfIndices(rw)

        selected_w.append(ts)
        selected_c.append(np.full(n_random, c, np.int32))

    # dimensions
    selected_w = np.concatenate(selected_w) # (n, len, dims)
    selected_w = np.transpose(selected_w, (0, 2, 1))

    meds, meds_id = compute_medoids(selected_w, np.concatenate(selected_c, axis=0))

    return np.transpose(meds[:,0,:,:], (0, 2, 1))

In [ ]:
# meds = obtain_medoids(ds, 500)

meds = np.load("./datasets/UCI-HAR/medoids.npz")

In [ ]:
fig, ax = plt.subplots(nrows=1)
[ax.plot(meds[0,:,d]) for d in range(6)]

In [ ]:
from s3ts.api.dms.har_datasets import DFDataset, DFDatasetCopy

In [ ]:
ds_df = DFDataset(ds, patterns=meds, w=0.1)

In [ ]:
del ds_df

In [ ]:
plot, ax = plt.subplots(nrows=2, ncols=3)

title = plot.text(0.5,0.85, "")

ax_images = []
for i in range(2):
    for j in range(3):
        if i==0:
            ax_images.append(ax[i, j].imshow(np.random.randn(64, 64), vmin=0, vmax=100))
        if i==1:
            ax_images.append(ax[i, j].imshow(np.random.randn(64, 64), vmin=0, vmax=100))

        ax_twin = ax[i, j].twinx()
        ax_twin.set_xlim([0, 64])
        ax_twin.set_ylim([0, 64])
        [ax_twin.plot(meds[i*3+j,:,h]*20 + 15, np.arange(64)) for h in range(6)]

for i in range(10000, 20000, 16):
    for col in range(3):
        df, ts, c = ds_df[i]
        for h in range(6):
            ax_images[h].set_data(df[h].T)

    title.set_text(f"Class {UCI_HAR_LABELS[c]}")
    display(plot)    
    clear_output(wait = True)